In [1]:
import numpy as np
import sys
import os
import neuro
import risp
import random
import time

import json

from encode import Chromagram
from encode import Encoder
from data_loader import DataLoader
import risp
import neuro
import eons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score


1.) Innitialize the neuroprocessor

In [2]:
risp_config = {
  "leak_mode": "all",
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "max_delay": 5,
  "discrete": False
}

proc = risp.Processor(risp_config) # RISP processor

net = neuro.Network() # Neuro network
net.set_properties(proc.get_network_properties()) # Set network properties


2.) Load the data

In [3]:
# First, split the data into training+validation and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(DataLoader().chroma_files, DataLoader().numerical_label_types, test_size=0.2, random_state=42)

# Now, split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# One-hot encode the labels
OneHot = OneHotEncoder(sparse_output=False)
y_train_encoded = OneHot.fit_transform(np.array(y_train).reshape(-1, 1))
y_val_encoded = OneHot.transform(np.array(y_val).reshape(-1, 1))
y_test_encoded = OneHot.transform(np.array(y_test).reshape(-1, 1))

In [4]:
print("Shape of y_test:", np.shape(y_test_encoded[0]))

Shape of y_test: (10,)


In [5]:
y_train_encoded[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

3.) create template for eons

In [6]:
n_inputs = 12
n_hidden = 100
n_outputs = 10
n_neurons = n_inputs + n_hidden + n_outputs
n_synapses = 200


moa = neuro.MOA()
moa.seed(42)

In [7]:
def create_neuron(neuron_id, net, moa):
    neuron = net.add_node(neuron_id)
    net.randomize_node_properties(moa, neuron)
    return neuron

In [8]:
for i in range(n_inputs):
    neuron = create_neuron(i, net, moa)
    neuron.set("Threshold",0.75)
    net.add_input(i)
    
for i in range(n_outputs):
    neuron = create_neuron(i+n_inputs, net, moa)
    neuron.set("Threshold",0.75)
    net.add_output(i)
    
for i in range(n_hidden):
    neuron = create_neuron(i+n_inputs+n_outputs, net, moa)

In [9]:
print("Neuron 0's input ID:", net.get_node(0).input_id)
print("Neuron 1's input ID:", net.get_node(1).input_id)

Neuron 0's input ID: 0
Neuron 1's input ID: 1


In [10]:
for i in range(n_synapses):
    source = random.randint(0,n_neurons-1)
    dest = random.randint(0,n_neurons-1)
    synapse = net.add_or_get_edge(source, dest)
    net.randomize_edge_properties(moa, synapse)

4.) set up the eons

In [12]:
eo_params =  {
    "starting_nodes": 60,
    "starting_edges": 60,
    "merge_rate": 0,
    "population_size": 50,
    "multi_edges": 0,
    "crossover_rate": 0.5,
    "mutation_rate": 0.9,
    "selection_type": "tournament",
    "tournament_size_factor": 0.3,
    "tournament_best_net_factor": 0.9,
    "random_factor": 0,
    "num_mutations": 5,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.7, "Delay": 0.3 },
    "num_best": 2,

    "add_node_rate": 0.55,
    "delete_node_rate": 0.45,
    "add_edge_rate": 0.55,
    "delete_edge_rate": 0.45,
    "node_params_rate": 2.5,
    "edge_params_rate": 2.5,
    "net_params_rate" : 0
}



In [39]:
def read_network(fn):
    with open(fn, "r") as f:
        s = f.read()
        j = json.loads(s)
        net = neuro.Network()
        net.from_json(j)
    return net

In [41]:
network = read_network("test_model.json")

In [13]:
print(network.num_edges())
print(network.num_nodes())

NameError: name 'network' is not defined

In [14]:
evolver = eons.EONS(eo_params)
evolver.set_template_network(net)

pop = evolver.generate_population(eo_params,42)

In [15]:
def get_prediction(proc, x):
    # Load the chroma data from the npy file
    encoder=Encoder(x)
    
    proc.clear_activity()
    proc.apply_spikes(encoder.spikes)
    proc.run(encoder.time_steps * encoder.num_frames * 4) # you might adjust this duration based on your needs
    
    # Decoding the output to get the predicted label. You might need to adjust this
    predicted_index = proc.output_count_max(n_outputs)[0]
    # Convert index to one-hot encoded format
    one_hot_prediction = np.zeros(n_outputs)
    one_hot_prediction[predicted_index] = 1.0
    
    return one_hot_prediction 


In [16]:
def fitness(proc, net, X, y):
    proc.load_network(net)
    
    # Set up output tracking
    for i in range(n_outputs):
        proc.track_neuron_events(i)
    
    y_predict = [get_prediction(proc, x) for x in X]
    return accuracy_score(y_predict, y)


In [17]:
max_generations = 100
vals = []

for generation in range(max_generations):
    start_time = time.time()  # Record the start time for this generation

    # Evaluate fitness of all networks in the population
    fitnesses = [fitness(proc, net.network, X_train, y_train_encoded) for net in pop.networks]
    
    # Track and print best fitness in the current generation
    best_fitness = max(fitnesses)
    vals.append(best_fitness)
    best_net = pop.networks[fitnesses.index(best_fitness)].network

    end_time = time.time()  # Record the end time for this generation
    elapsed_time = time.strftime("%H:%M:%S", time.gmtime(end_time - start_time))

    print(f"Generation {generation + 1}, Best Fitness: {best_fitness:.4f}, Time: {elapsed_time}, Max_Nodes: {best_net.num_nodes()}")
    
    # Produce the next generation based on the current population's fitness
    pop = evolver.do_epoch(pop, fitnesses, eo_params)

# Optionally, you can evaluate and print the performance of the best network on the test set here.

print("Evolution finished!")
print("Best fitness:", best_fitness)

Generation 1, Best Fitness: 0.0999, Time: 00:05:41, Max_Nodes: 72
Generation 2, Best Fitness: 0.1039, Time: 00:05:06, Max_Nodes: 73
Generation 3, Best Fitness: 0.1056, Time: 00:05:35, Max_Nodes: 72
Generation 4, Best Fitness: 0.1056, Time: 00:06:22, Max_Nodes: 74
Generation 5, Best Fitness: 0.1056, Time: 00:05:22, Max_Nodes: 75
Generation 6, Best Fitness: 0.1067, Time: 00:04:58, Max_Nodes: 73
Generation 7, Best Fitness: 0.1067, Time: 00:04:57, Max_Nodes: 73
Generation 8, Best Fitness: 0.1067, Time: 00:05:21, Max_Nodes: 74
Generation 9, Best Fitness: 0.1067, Time: 00:04:57, Max_Nodes: 72
Generation 10, Best Fitness: 0.1073, Time: 00:04:38, Max_Nodes: 72


KeyboardInterrupt: 

In [18]:
best_net = pop.networks[fitnesses.index(best_fitness)].network
validation_net = fitness(proc, best_net, X_val, y_val_encoded)

In [42]:
validation_net

0.0958904109589041

In [32]:
network_json = best_net.as_json().to_python()
filename = "test_model.json"

In [35]:
type(network_json)

dict

In [36]:
network_json

{'Associated_Data': {},
 'Edges': [{'from': 57, 'to': 44, 'values': [-0.8643327993024915, 1]},
  {'from': 75, 'to': 10, 'values': [-0.9861522447285597, 1]},
  {'from': 57, 'to': 32, 'values': [0.5715615622633048, 4]},
  {'from': 52, 'to': 59, 'values': [-0.38097826051082884, 1]},
  {'from': 37, 'to': 55, 'values': [-0.09567161442145511, 3]},
  {'from': 76, 'to': 51, 'values': [0.23729130980495627, 1]},
  {'from': 4, 'to': 8, 'values': [-0.6829439927085639, 3]},
  {'from': 40, 'to': 50, 'values': [0.5198554423451087, 2]},
  {'from': 70, 'to': 68, 'values': [0.31653049153195, 3]},
  {'from': 34, 'to': 10, 'values': [-0.5615201996549778, 4]},
  {'from': 3, 'to': 40, 'values': [-0.6976199293736415, 1]},
  {'from': 26, 'to': 60, 'values': [0.9600044335145514, 3]},
  {'from': 68, 'to': 46, 'values': [-0.12767720947220862, 1]},
  {'from': 59, 'to': 45, 'values': [-0.17192908729704304, 1]},
  {'from': 81, 'to': 2, 'values': [-0.39840676807761355, 1]},
  {'from': 28, 'to': 68, 'values': [0.6568

In [37]:
with open(filename, "w") as f:
    json.dump(network_json, f)